Dyck Path Method

In [176]:
class dyckpaths:
    
    def __init__(self, n, one_count, zero_count):
        
        self.left = None
        self.right = None
        
        if one_count < n:
            self.left = dyckpaths(n, one_count + 1, zero_count)
        if zero_count < one_count:
            self.right = dyckpaths(n, one_count, zero_count + 1)
            
            
    def get_dyckpaths(self, root, paths):
        
        if self.left:
            self.left.get_dyckpaths(root + [1], paths)
            
        if self.right:
            self.right.get_dyckpaths(root + [0], paths)
            
        if self.left == None and self.right == None:
            paths.append(root)

paths = []
dyckpaths(4, 0, 0).get_dyckpaths([], paths)
print(paths)

[[1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 0, 0], [1, 1, 1, 0, 0, 1, 0, 0], [1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 1, 1, 0, 0, 0], [1, 1, 0, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 1, 0, 0], [1, 1, 0, 0, 1, 0, 1, 0], [1, 0, 1, 1, 1, 0, 0, 0], [1, 0, 1, 1, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1, 1, 0, 0], [1, 0, 1, 0, 1, 0, 1, 0]]


In [178]:
exps = []

for path in paths:
    exp = ''
    num = -1
    for p in path:
        if p == 1:
            num += 1
            exp += f'({num}'
        if p == 0:
            exp += ')'
            
    exps.append(exp)
    
print(exps)

['(0(1(2(3))))', '(0(1(2)(3)))', '(0(1(2))(3))', '(0(1(2)))(3)', '(0(1)(2(3)))', '(0(1)(2)(3))', '(0(1)(2))(3)', '(0(1))(2(3))', '(0(1))(2)(3)', '(0)(1(2(3)))', '(0)(1(2)(3))', '(0)(1(2))(3)', '(0)(1)(2(3))', '(0)(1)(2)(3)']


In [168]:
class star_form:
    
    def __init__(self, string):
        self.string = string
        
    def __call__(self, other):
        inp = str(other)
#         new_string = "("+ self.string + " * " + star_form(other).string + ")"
        return inp


In [169]:

eg = 'A(Z)'
print(eval(eg))

<string>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?


TypeError: 'int' object is not callable

In [244]:
def find_input(multiplier, exp):

    right = 0
    left= 0
    inputs = False
    number = False

    leng = len(multiplier)

    inp = ""
    num = ""


    for i, char in enumerate(exp):
            
        
        if exp[i-leng:i] == multiplier:
            inputs = True
            mult = multiplier
            right = 0
            left = 0
            
        if exp[i-2-leng:i] == "(" + multiplier + ")":
            inp = ""
            mult = "(" + multiplier + ")"
            right = 0
            left = 0
                    
        if inputs and left == 0 and char.isnumeric():
            number = True
            num += char

        if number and left == 0 and not char.isnumeric():
            inp = num
            return inp

        if char == "(":
            left += 1
        if char == ")":
            right += 1

        if inputs and left >= right: 
            inp += char
            mult += char

        if inputs and not number and left == right:
            if inp == "(":
                return None
            return inp, mult

In [253]:
a, b = find_input("0", "0(5)")
print(type(a), b)

<class 'str'> 0(5)


In [256]:
def get_structure(exp):
    '''
    Splits an expression into its functions (in order) and its composition structure.
    '''
    new_exp = ""
    SKI = []
    j = 0
    for char in exp:
        if char == "S" or char == "K" or char == "I":
            new_exp += f"{j}"
            SKI.append(char)
            j += 1
        else:
            new_exp += char

    return new_exp, SKI 

In [257]:
import re

def subin_to_exp(exp, SKI):

    l = len(SKI) - 1
    for i, s in enumerate(SKI):
        exp = re.sub(fr"{l - i}", fr"{SKI[l - i]}", exp)
    
    return exp

In [276]:
### defining SKIsimplify()
import re

def SKIsimplify(SKI_exp):
    
    exp, SKI = get_structure(SKI_exp)
    
    only_S = False
    
    shortest_exp = (len(exp), exp, SKI)
    
    it = 0
    j = -1
    while j < len(SKI)-1:
        it += 1
        j +=1
        
        
        prod = f"{j}"
        # identifying the functions curried inputs
        inputs = []
        prods = []
        while len(inputs) <= 3:
            
            if find_input(prod, exp) == None:
                break
            else:
                inp, prod = find_input(prod, exp)

            inputs.append(inp)
            prods.append(prod)

        ## Create inputss, its just inputs but has a \ before every symbol so re.sub can read it
        inputss = []
        for inp in inputs:
            inps = ""
            for i in inp:
                if i == "(" or i == ")":
                    i = re.sub(fr"\{i}", fr"\\{i}", i)
                inps += i
            inputss.append(inps)
            
        prodss = []
        for prod in prods:
            pro = ""
            for i in prod:
                if i == "(" or i == ")":
                    i = re.sub(fr"\{i}", fr"\\{i}", i)
                pro += i
            prodss.append(pro)
        
        ### Performing the action of the combinators in the expression
        
        if SKI[int(j)] == "K" and len(inputs) >= 2:

            exp = re.sub(prodss[1], inputs[0], exp)
            j = -1
            only_S = False

        elif SKI[int(j)] == "I" and len(inputs) >= 1:

            exp = re.sub(prodss[0], inputs[0], exp)
            j = -1
            only_S = False
                    
        elif SKI[int(j)] == "S" and len(inputs) >= 3 and only_S:

            exp = re.sub(prodss[2], 
                         fr"{inputs[0]}({inputs[2]})({inputs[1]}({inputs[2]}))", exp)
            j = -1
            only_S = False
        
        if j == len(SKI) - 1 and not only_S:
            j = -1
            only_S = True


        ## Renumbering our new expression, and finding the new order of S, K, Is
        nums = []
        new_exp = ""
        count = 0
        num = ""
        number = False
        for i, char in enumerate(exp):
            if char.isnumeric() and len(exp) == 1:
                number = True
                new_exp += "0"
                nums.append(char)
            if char.isnumeric() and len(exp) != 1:
                number = True
                num += char
            if number and not char.isnumeric():
                number = False
                new_exp += f"{count}"
                nums.append(num)
                num = ""
                count += 1
            if not number:
                new_exp += char

        ## Reording our SKI list according to the new combination after the action of the combinators
        new_SKI = []
        for no in nums:
            new_SKI.append(SKI[int(no)])
        
        ## Updating our expression and SKI list
        SKI = new_SKI
        exp = new_exp
        
        ## update the shortest new_exp so far - will be returned if the combo is volatile
        if len(exp) < shortest_exp[0]:
            shortest_exp = (len(exp), exp, SKI)
        if it > 100:
            return subin_to_exp(shortest_exp[1], shortest_exp[2]), "vol"
        
    return subin_to_exp(exp, SKI), "con"

In [277]:
SKIsimplify("((S(I))(I))((S(I))(I))")

('(((S(I))(I)))((((S(I))(I))))', 'con')